<a href="https://colab.research.google.com/github/jonatasbx/Curso-CDIA/blob/main/DF_tratamento_sus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Análise de custos de internação no DF

### Para a análise pretendida, a base utilizada foi a do mês de abril de 2024 da UF DF

### Problema: De acordo com o histórico de dados qual a previsão do custo de internação de uma pessoa no DF em abril 2024

---



#### Importação das bibliotecas

---



In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Importação da base

---



In [69]:
pd.set_option('display.float_format', '{:.2f}'.format)
df_sus = pd.read_csv('BancoDF.csv')

In [70]:
df_sus.head()

,ESPEC,IDENT,MUNIC_RES,SEXO,UTI_MES_IN,UTI_MES_AN,UTI_MES_AL,UTI_MES_TO,MARCA_UTI,DIAR_ACOM,QT_DIARIAS,PROC_REA,VAL_TOT,DIAG_PRINC,COBRANCA,NATUREZA,NAT_JUR,GESTAO,IDADE,DIAS_PERM,NACIONAL,CAR_INT,GESTRISCO,COMPLEX,FINANC,RACA_COR,ETNIA,SEQUENCIA,DIAGSEC1
0,1,1,530010,1,0,0,0,29,75,0,2,408050012,18773.07,R02,41,0,1023,2,59,30,10,2,1,2,6,3,0,505,NaN
1,1,1,521760,1,0,0,0,7,75,0,16,407030123,5948.23,D731,15,0,1023,2,32,23,10,2,1,2,6,2,0,506,NaN
2,1,1,521760,1,0,0,0,0,0,0,2,415010012,3735.12,K562,27,0,1023,2,50,1,10,2,1,2,6,3,0,507,K564
3,1,1,521760,1,0,0,0,6,75,0,2,415010012,6570.71,K564,27,0,1023,2,50,7,10,2,1,2,6,3,0,509,K650
4,1,1,521760,1,0,0,0,2,75,0,2,415010012,4807.07,K564,27,0,1023,2,50,3,10,2,1,2,6,3,0,510,K822


#### Tratamento da base e seleção das variáveis de interesse

---



In [71]:
pd.options.display.max_columns = 113
df_sus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22390 entries, 0 to 22389
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ESPEC       22390 non-null  int64  
 1   IDENT       22390 non-null  int64  
 2   MUNIC_RES   22390 non-null  int64  
 3   SEXO        22390 non-null  int64  
 4   UTI_MES_IN  22390 non-null  int64  
 5   UTI_MES_AN  22390 non-null  int64  
 6   UTI_MES_AL  22390 non-null  int64  
 7   UTI_MES_TO  22390 non-null  int64  
 8   MARCA_UTI   22390 non-null  int64  
 9   DIAR_ACOM   22390 non-null  int64  
 10  QT_DIARIAS  22390 non-null  int64  
 11  PROC_REA    22390 non-null  int64  
 12  VAL_TOT     22390 non-null  float64
 13  DIAG_PRINC  22390 non-null  object 
 14  COBRANCA    22390 non-null  int64  
 15  NATUREZA    22390 non-null  int64  
 16  NAT_JUR     22390 non-null  int64  
 17  GESTAO      22390 non-null  int64  
 18  IDADE       22390 non-null  int64  
 19  DIAS_PERM   22390 non-nul

Tratamento das variáveis DIAG_PRINC e DIAGSEC1

As variáveis trazem o codigo cid composto por uma letra que representa a classe e um codigo numérico que representa a doença. Trataremos as variáveis para representarem apenas as classes de doenças.

In [72]:
df_sus["DIAGSEC1"].unique()

array([nan, 'K564', 'K650', 'K822', 'Y349', 'X589', 'Y899', 'E669',
       'K450', 'S367', 'Z515', 'Y000', 'Y849', 'W199', 'W999', 'W009',
       'V299', 'J459', 'D571', 'G809', 'V289', 'M702', 'A90 ', 'N189',
       'I64 ', 'I500', 'V129', 'W019', 'W329', 'D410', 'M328', 'V874',
       'J039', 'J210', 'F03 ', 'H659', 'W189', 'V899', 'X599', 'W179',
       'V224', 'Y049', 'V180', 'W105', 'V234', 'W209', 'V239', 'W449',
       'C341', 'F168', 'K830', 'J960', 'W777', 'I10 ', 'R568', 'X649',
       'V029', 'N390', 'J159', 'J189', 'V298', 'I420', 'I509', 'V229',
       'I51 ', 'T369', 'X228', 'N459', 'W000', 'W290', 'Y838', 'X789',
       'A509', 'X220', 'X200', 'X229', 'L249', 'J21 ', 'V429', 'B342',
       'I50 ', 'W419', 'V199', 'V486', 'S062', 'G049', 'M861', 'X400',
       'X100', 'W010', 'C539', 'W499', 'I739', 'X999', 'X209', 'T909',
       'N488', 'X840', 'Y848', 'V092', 'X449', 'J18 ', 'W069', 'I619',
       'W319', 'I702', 'W109', 'V093', 'X640', 'W190', 'Y189', 'V249',
       'B

In [73]:
df_sus['DIAGSEC1'] = df_sus['DIAGSEC1'].astype(str).str[0]

In [74]:
df_sus['DIAGSEC1'].unique()

array(['n', 'K', 'Y', 'X', 'E', 'S', 'Z', 'W', 'V', 'J', 'D', 'G', 'M',
       'A', 'N', 'I', 'F', 'H', 'C', 'R', 'T', 'L', 'B', 'O', 'Q', 'P'],
      dtype=object)

Como não há nos dados CID iniciando em U, usaremos esse codigo para os Cids em não preenchidos da variável.

In [75]:
df_sus['DIAGSEC1'] = df_sus['DIAGSEC1'].replace('n','U')

In [76]:
df_sus['DIAGSEC1'].unique()

array(['U', 'K', 'Y', 'X', 'E', 'S', 'Z', 'W', 'V', 'J', 'D', 'G', 'M',
       'A', 'N', 'I', 'F', 'H', 'C', 'R', 'T', 'L', 'B', 'O', 'Q', 'P'],
      dtype=object)

In [77]:
from sklearn.preprocessing import LabelEncoder

le_diagsec = LabelEncoder()
df_sus['DIAGSEC1'] = le_diagsec.fit_transform(df_sus['DIAGSEC1'])
df_sus['DIAGSEC1'].unique()

array([20, 10, 24, 23,  4, 18, 25, 22, 21,  9,  3,  6, 12,  0, 13,  8,  5,
        7,  2, 17, 19, 11,  1, 14, 16, 15])

In [78]:
diagsec1_mapping = dict(zip(le_diagsec.classes_, le_diagsec.transform(le_diagsec.classes_)))
print("Dicionário de encoding para DIAGSEC1:")
print(diagsec1_mapping)

Dicionário de encoding para DIAGSEC1:
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}


Agora o mesmo procedimento será aplicado para a variável DIAG_PRINC. Com a diferença que esta variável não apresenta valores em branco

In [79]:
le_diagprinc = LabelEncoder()

In [80]:
df_sus["DIAG_PRINC"].unique()

array(['R02 ', 'D731', 'K562', ..., 'M16 ', 'Q898', 'K605'], dtype=object)

In [81]:
df_sus['DIAG_PRINC'] = df_sus['DIAG_PRINC'].astype(str).str[0]

In [82]:
df_sus["DIAG_PRINC"].unique()

array(['R', 'D', 'K', 'A', 'O', 'E', 'J', 'T', 'C', 'L', 'H', 'N', 'S',
       'Z', 'Q', 'I', 'G', 'M', 'F', 'P', 'B'], dtype=object)

In [83]:
df_sus['DIAG_PRINC'] = le_diagprinc.fit_transform(df_sus['DIAG_PRINC'])
df_sus['DIAG_PRINC'].unique()

array([17,  3, 10,  0, 14,  4,  9, 19,  2, 11,  7, 13, 18, 20, 16,  8,  6,
       12,  5, 15,  1])

In [84]:
diagprinc_mapping = dict(zip(le_diagprinc.classes_, le_diagprinc.transform(le_diagprinc.classes_)))
print("\nDicionário de encoding para DIAG_PRINC:")
print(diagprinc_mapping)


Dicionário de encoding para DIAG_PRINC:
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'Z': 20}


In [88]:
df_sus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22390 entries, 0 to 22389
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ESPEC       22390 non-null  int64  
 1   IDENT       22390 non-null  int64  
 2   MUNIC_RES   22390 non-null  int64  
 3   SEXO        22390 non-null  int64  
 4   UTI_MES_IN  22390 non-null  int64  
 5   UTI_MES_AN  22390 non-null  int64  
 6   UTI_MES_AL  22390 non-null  int64  
 7   UTI_MES_TO  22390 non-null  int64  
 8   MARCA_UTI   22390 non-null  int64  
 9   DIAR_ACOM   22390 non-null  int64  
 10  QT_DIARIAS  22390 non-null  int64  
 11  PROC_REA    22390 non-null  int64  
 12  VAL_TOT     22390 non-null  float64
 13  DIAG_PRINC  22390 non-null  int64  
 14  COBRANCA    22390 non-null  int64  
 15  NATUREZA    22390 non-null  int64  
 16  NAT_JUR     22390 non-null  int64  
 17  GESTAO      22390 non-null  int64  
 18  IDADE       22390 non-null  int64  
 19  DIAS_PERM   22390 non-nul

#### Todas as variáveis são numéricas, iniciando análise exploratória de dados

---

In [89]:
df_sus.describe()

,ESPEC,IDENT,MUNIC_RES,SEXO,UTI_MES_IN,UTI_MES_AN,UTI_MES_AL,UTI_MES_TO,MARCA_UTI,DIAR_ACOM,QT_DIARIAS,PROC_REA,VAL_TOT,DIAG_PRINC,COBRANCA,NATUREZA,NAT_JUR,GESTAO,IDADE,DIAS_PERM,NACIONAL,CAR_INT,GESTRISCO,COMPLEX,FINANC,RACA_COR,ETNIA,SEQUENCIA,DIAGSEC1
count,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00,22390.00
mean,3.66,1.02,522213.95,2.18,0.00,0.00,0.00,0.59,4.89,3.58,5.42,337689769.48,1518.94,9.38,21.26,0.00,1622.15,2.00,33.31,5.79,10.24,1.87,1.00,2.06,5.91,2.58,0.03,11817.73,19.71
std,5.00,0.28,41598.11,0.98,0.00,0.00,0.00,3.46,18.76,6.40,7.94,51026531.25,4126.34,6.46,15.21,0.00,975.18,0.00,24.55,8.49,5.90,0.38,0.04,0.24,0.41,0.80,2.31,6727.96,2.75
min,1.00,1.00,110002.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,201010143.00,0.00,0.00,11.00,0.00,1023.00,2.00,0.00,0.00,10.00,1.00,0.00,2.00,4.00,1.00,0.00,283.00,0.00
25%,2.00,1.00,530010.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,303070110.00,287.76,5.00,12.00,0.00,1023.00,2.00,11.00,2.00,10.00,2.00,1.00,2.00,6.00,3.00,0.00,5921.25,20.00
50%,3.00,1.00,530010.00,3.00,0.00,0.00,0.00,0.00,0.00,2.00,3.00,303170140.00,564.93,7.00,15.00,0.00,1023.00,2.00,30.00,3.00,10.00,2.00,1.00,2.00,6.00,3.00,0.00,11893.50,20.00
75%,7.00,1.00,530010.00,3.00,0.00,0.00,0.00,0.00,0.00,4.00,6.00,407020136.00,1019.93,15.00,26.00,0.00,3069.00,2.00,52.00,6.00,10.00,2.00,1.00,2.00,6.00,3.00,0.00,17724.75,20.00
max,87.00,5.00,530010.00,3.00,0.00,0.00,0.00,75.00,99.00,95.00,104.00,506020118.00,130211.83,20.00,64.00,0.00,3999.00,2.00,99.00,103.00,277.00,6.00,1.00,3.00,6.00,5.00,241.00,23373.00,25.00


In [87]:
sih_sus_tratada = sih_sus.drop([
    'CGC_HOSP',
    'CNPJ_MANT',
    'AUD_JUST',
    'SIS_JUST',
    'FAEC_TP',
    'GESTOR_DT',
    'INFEHOSP',
    'CID_NOTIF',
    'NUM_PROC',
    'DIAG_PRINC',
    'REMESSA',
    'CEP',
    'MUNIC_RES',
    'NASC',
    'DIAR_ACOM',
    'SEXO',
    #'PROC_SOLIC',
    #'VAL_UTI',
    #'VAL_SH',
    'DT_INTER',
    'DT_SAIDA',
    'COBRANCA',
    'RUBRICA',
    'MUNIC_MOV',
    'NACIONAL',
    'CAR_INT',
    'HOMONIMO',
    'NUM_FILHOS',
    'INSTRU',
    'INSC_PN',
    'SEQ_AIH5',
    'CBOR',
    'CNAER',
    'VINCPREV',
    'GESTOR_COD',
    'GESTOR_TP',
    'GESTOR_CPF',
    'CNES',
    'CID_ASSO',
    'CID_MORTE',
    'REGCT',
    'SEQUENCIA',
    'VAL_SH_FED',
    'VAL_SP_FED',
    'VAL_SH_GES',
    'VAL_SP_GES',
    'VAL_UCI',
    'DIAS_PERM',
    'US_TOT',
    'MES_CMPT',
    'ANO_CMPT',
    'ESPEC',
    'NATUREZA',
    'NAT_JUR',
    'COD_IDADE',
   # 'DIAG_SECUN',
    'RACA_COR',
    'ETNIA',
    'UF_ZI',
    'MORTE',
    'DIAGSEC1',
    'DIAGSEC2',
    'DIAGSEC3',
    'DIAGSEC4',
    'DIAGSEC5',
    'DIAGSEC6',
    'DIAGSEC7',
    'DIAGSEC8',
    'DIAGSEC9',
    'TPDISEC1',
    'CPF_AUT'], axis=1)


NameError: name 'sih_sus' is not defined

In [ ]:
sih_sus_tratada.isnull().sum()

In [ ]:
sih_sus_tratada.info()

### Todas as variáveis com valores nulos foram retiradas assim, como as variáveis categóricas.

## Inicio da EDA

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))
plt.hist(sih_sus_tratada['IDADE'], bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Idade')
plt.ylabel('Frequência')
plt.title('Histograma da Idade')
plt.show()

plt.figure(figsize=(12, 10))

correlation_matrix = sih_sus_tratada.corr()

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

plt.title('Matriz de Correlação (Metade Superior)')
plt.show()


### Início da modelagem de dados, testaremos os modelos de Regressão Linear, KNN com 5 e 10 vizinhos e Decision Tree

### Regressão Linear

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
X = sih_sus_tratada.drop('VAL_TOT', axis=1)
y = sih_sus_tratada['VAL_TOT']

In [ ]:
regressao_linear = LinearRegression()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
regressao_linear.fit(X_train, y_train)

In [ ]:
tabela_coef_rl = pd.DataFrame({'Variável': X.columns, 'Coeficiente': regressao_linear.coef_})

In [ ]:
tabela_coef_rl

In [ ]:
y_pred = regressao_linear.predict(X_test)

In [ ]:
resultado = pd.DataFrame({'Valor Real': y_test, 'Valor Predito': y_pred})

In [ ]:
resultado

In [ ]:
r2 = r2_score(y_test, y_pred)
print(f'R2 :{r2}')
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')
mae = mean_absolute_error(y_test, y_pred)
print(f'MAE: {mae}')

### O modelo mostrou conseguir explicar 0,68 (r2)

KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn5 = KNeighborsRegressor(n_neighbors=5)

In [ ]:
knn5.fit(X_train, y_train)

In [ ]:
knn5_pred = knn5.predict(X_test)

In [ ]:
knn5_resultado = pd.DataFrame({'Valor Real': y_test, 'Valor Predito': knn5_pred})

In [ ]:
knn5_resultado

In [ ]:
knn5_r2 = r2_score(y_test, knn5_pred)
print(f'R2 :{knn5_r2}')
knn5_mse = mean_squared_error(y_test, knn5_pred)
print(f'MSE: {knn5_mse}')
knn5_mae = mean_absolute_error(y_test, knn5_pred)
print(f'MAE: {knn5_mae}')

## O resultado para o KNN foi bem pior que o resultado da regressão linear

In [ ]:
knn10 = KNeighborsRegressor(n_neighbors=10)

In [ ]:
knn10.fit(X_train, y_train)

In [ ]:
knn10_pred = knn10.predict(X_test)

In [ ]:
knn10_resultado = pd.DataFrame({'Valor Real': y_test, 'Valor Predito': knn10_pred})

In [ ]:
knn10_r2 = r2_score(y_test, knn10_pred)
print(f'R2 :{knn10_r2}')
knn10_mse = mean_squared_error(y_test, knn10_pred)
print(f'MSE: {knn10_mse}')
knn10_mae = mean_absolute_error(y_test, knn10_pred)
print(f'MAE: {knn10_mae}')

## O resultado melhorou um pouco com 10 vizinhos, mas continua com uma performance abaixo da regressão linear

### Árvore de decisão

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dt = DecisionTreeRegressor(max_depth=10)

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
dt_pred = dt.predict(X_test)

In [ ]:
dt_resultado = pd.DataFrame({'Valor Real': y_test, 'Valor Predito': dt_pred})

In [ ]:
dt_resultado

In [ ]:
dt_r2 = r2_score(y_test, dt_pred)
print(f'R2 :{dt_r2}')
dt_mse = mean_squared_error(y_test, dt_pred)
print(f'MSE: {dt_mse}')
dt_mae = mean_absolute_error(y_test, dt_pred)
print(f'MAE: {dt_mae}')

## Dentre os três modelos de regressão o que apresentou melhor resultado foi a regressão linear